# Ground-roll attenuation model inference

Required imports:

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import segyio
from tqdm import tqdm
import sh

sys.path.append('..')

from seismicpro.batchflow import Dataset, DatasetIndex, Pipeline, V

from seismicpro.src import (SeismicBatch, FieldIndex, TraceIndex,
                            seismic_plot, spectrum_plot, merge_segy_files)
from seismicpro.models import FieldMetrics
from seismicpro.src.file_utils import write_segy_file
from seismicpro.src.seismic_batch import FILE_DEPENDEND_COLUMNS

from unet import u_net, conv_block

%env CUDA_VISIBLE_DEVICES=7

env: CUDA_VISIBLE_DEVICES=7


Index raw dataset:

In [2]:
path_raw = '/notebooks/egor/noise_dataset_1/DN01_shots_for_lift_well.sgy'

index = TraceIndex(name='raw', extra_headers='all', path=path_raw)
index.head()

,HighCutSlope,TraceWeightingFactor,CDP_X,SourceSurfaceElevation,TraceIdentifier,ShotPoint,SubWeatheringVelocity,SourceY,SourceType,DayOfYear,...,SourceGroupScalar,SourceMeasurementExponent,GeophoneGroupNumberLastTraceOrigField,CDP,TRACE_SAMPLE_COUNT,GainType,NStackedTraces,ElevationScalar,TRACE_SEQUENCE_FILE,file_id
,,,,,,,,,,,...,,,,,,,,,raw,raw
0,0,0,502930,37,0,37281,0,6700517,0,43,...,1,0,0,1745712,3000,0,0,1,1,/notebooks/egor/noise_dataset_1/DN01_shots_for...
1,0,0,502880,37,0,37281,0,6700517,0,43,...,1,0,0,1745710,3000,0,0,1,2,/notebooks/egor/noise_dataset_1/DN01_shots_for...
2,0,0,502905,37,0,37281,0,6700517,0,43,...,1,0,0,1745711,3000,0,0,1,3,/notebooks/egor/noise_dataset_1/DN01_shots_for...
3,0,0,502830,37,0,37281,0,6700517,0,43,...,1,0,0,1745708,3000,0,0,1,4,/notebooks/egor/noise_dataset_1/DN01_shots_for...
4,0,0,502855,37,0,37281,0,6700517,0,43,...,1,0,0,1745709,3000,0,0,1,5,/notebooks/egor/noise_dataset_1/DN01_shots_for...


Set path to the model:

In [5]:
model_path = '/notebooks/egor/gazprom-neft/SeismicPro/models/Ground-roll_attenuation/Attention_model/tf_model2.ckpt'

Restore the model:

In [4]:
config = tf.ConfigProto()
graph = tf.Graph()
with graph.as_default():
    trace_raw = tf.placeholder('float', shape=(None, 3000, 1), name='trace_in')
    targets = tf.placeholder('float', shape=(None, 3000, 1), name='target')
    alpha = tf.placeholder('float', name='alpha')
    beta = tf.placeholder('float', name='beta')
    learning_rate = tf.placeholder('float', name='learning_rate')
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    #Attention branch
    with tf.variable_scope("attention_scope"):
        out_attention = u_net(trace_raw, depth=3, filters=8, kernel_size=3,
                              activation='elu', is_training=is_training)
        out_attention = conv_block(out_attention, 'ca', filters=1, kernel_size=3,
                                   activation='sigmoid')
        print('attention', out_attention.get_shape())

    attention_sum = tf.reduce_sum(out_attention, axis=1, keepdims=True)
    print('attention_sum', attention_sum.get_shape())

    #Define a domain for sigmoid function
    sigm_x = tf.fill(tf.shape(out_attention), 0.0)
    arange = tf.range(0, tf.cast(tf.shape(sigm_x)[1], 'float'), dtype='float')
    arange = tf.expand_dims(arange, axis=-1)
    sigm_x = sigm_x - arange + attention_sum
    print('sigm_x', sigm_x.get_shape())
    
    #Apply sigmoid function to the above obtained domain
    attention_sigmoid = tf.sigmoid(sigm_x)
    print('attention_sigmoid', attention_sigmoid.get_shape())
    
    #Main branch
    with tf.variable_scope("main_scope"):
        out_main = u_net(trace_raw, depth=5, filters=16, kernel_size=7,
                         activation='elu', is_training=is_training)
        out_main = conv_block(out_main, 'c', filters=1, kernel_size=3)
    print('out_main', out_main.get_shape())
    
    #Get a model output that is a superposition of raw input and main branches
    #according to attention mask
    predictions = trace_raw * attention_sigmoid + out_main * (1 - attention_sigmoid)

    #Loss compontents
    noise_loss = tf.losses.absolute_difference(targets, predictions)
    cone_loss = alpha * tf.reduce_mean(1 - attention_sigmoid)
    sigm_loss = beta * tf.losses.absolute_difference(attention_sigmoid, out_attention)
    
    #Loss for attention mask and denoising
    total_loss = noise_loss + sigm_loss + cone_loss
#     total_loss = tf.clip_by_value(total_loss, 0, .1)

    #Weights for cone noise loss
    weights = (1 - attention_sigmoid) / tf.reduce_sum(1 - attention_sigmoid)
    print('weights', weights.get_shape())

    #Denoising loss within ground-roll cone
    cone_noise_loss = tf.losses.absolute_difference(targets, predictions,
                                                    reduction=tf.losses.Reduction.SUM,
                                                    weights=weights)
#     cone_noise_loss = tf.clip_by_value(cone_noise_loss, 0, .1)

    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    main_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                  scope='main_scope')
    attention_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                       scope='attention_scope')

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        step_attention = optimizer.minimize(total_loss, var_list=attention_vars)
        step_main = optimizer.minimize(total_loss, var_list=main_vars)
        step_cone = optimizer.minimize(cone_noise_loss, var_list=main_vars)
    
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, model_path)

input (?, 3000, 1)
conv_block_0 (?, 3000, 8)
pool_0 (?, 1500, 8)
conv_block_1 (?, 1500, 16)
pool_1 (?, 750, 16)
conv_block_2 (?, 750, 32)
pool_2 (?, 375, 32)
bottom_conv_block_3 (?, 375, 64)
up_2 (?, 750, 64)
concat_3 (?, 750, 96)
up_conv_block_3 (?, 750, 32)
up_1 (?, 1500, 32)
concat_2 (?, 1500, 48)
up_conv_block_2 (?, 1500, 16)
up_0 (?, 3000, 16)
concat_1 (?, 3000, 24)
up_conv_block_1 (?, 3000, 8)
attention (?, 3000, 1)
attention_sum (?, 1, 1)
sigm_x (?, 3000, 1)
attention_sigmoid (?, 3000, 1)
input (?, 3000, 1)
conv_block_0 (?, 3000, 16)
pool_0 (?, 1500, 16)
conv_block_1 (?, 1500, 32)
pool_1 (?, 750, 32)
conv_block_2 (?, 750, 64)
pool_2 (?, 375, 64)
conv_block_3 (?, 375, 128)
pool_3 (?, 188, 128)
conv_block_4 (?, 188, 256)
pool_4 (?, 94, 256)
bottom_conv_block_5 (?, 94, 512)
up_4 (?, 188, 512)
concat_5 (?, 188, 768)
up_conv_block_5 (?, 188, 256)
up_3 (?, ?, 256)
concat_4 (?, 375, 384)
up_conv_block_4 (?, 375, 128)
up_2 (?, 750, 128)
concat_3 (?, 750, 192)
up_conv_block_3 (?, 750, 64

Set path to temporary dumps:

In [6]:
tmp_dump_path = '/data/NA/tmp2'

Run inference:

In [ ]:
batch_size = 2000
run_set = Dataset(index, SeismicBatch)

def dump_single_segy(data, df, samples, path):
    """Dump data to segy file."""
    df.reset_index(drop=True, inplace=True)
    headers = list(set(df.columns.levels[0]) - set(FILE_DEPENDEND_COLUMNS))
    segy_headers = [h for h in headers if hasattr(segyio.TraceField, h)]
    df = df[segy_headers]
    df.columns = df.columns.droplevel(1)
    write_segy_file(data, df, samples, path)

for k in tqdm(range(1 + len(run_set) // batch_size)):
    batch = (run_set.next_batch(batch_size, n_epochs=1)
             .load(components=('raw',), fmt='segy', tslice=np.arange(3000)))

    x = np.expand_dims(np.vstack(batch.raw), -1)

    res = sess.run(predictions,
                   feed_dict={trace_raw: x, is_training: False})
    
    df = batch.index.get_df(reset=False)
    samples = batch.meta['raw']['samples']
    dump_single_segy(res, df, samples, os.path.join(tmp_dump_path, str(k) + '.sgy'))

 22%|██▏       | 819/3743 [38:57<2:19:50,  2.87s/it]

Set path to merged file:

In [ ]:
output_path = '/notebooks/egor/noise_dataset_1/merged_unetatt_well.sgy'

Merge temporary dumps into single file:

In [ ]:
merge_segy_files(output_path=output_path, extra_headers='all',
                 path=os.path.join(tmp_dump_path, '*.sgy'))
sh.rm(sh.glob(os.path.join(tmp_dump_path, '*')))

Done!